In [1]:
import jax.numpy as jnp
from jax import vmap, grad

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
#V = np.load('/content/drive/MyDrive/nn-reaxff/v-au.npy')
#nrg = np.load('/content/drive/MyDrive/nn-reaxff/nrg-au.npy')
V = np.load('/content/drive/MyDrive/boes-pd-o/v-pd.npy')
nrg = np.load('/content/drive/MyDrive/boes-pd-o/nrg-pd.npy')

In [7]:
from jax import jacobian, vmap

# Define covariance function
def mycov(xx_vals1, xx_vals2):
  return outputscale*jnp.exp(-1/2*((xx_vals1-xx_vals2)/lengthscale)**2)

#def mycov(xx_vals1, xx_vals2):
#  return outputscale*xx_vals1*xx_vals2

#define derivatives
v_grad = vmap(jacobian(mycov,1),0)
v_grad2 = vmap(jacobian(mycov,0),0)
ffppcov = vmap(jacobian(jacobian(mycov,1),1),0)
ffppcov2 = vmap(jacobian(jacobian(mycov,0),0),0)
fpfpcov = vmap(jacobian(jacobian(mycov,0),1),0)
hess_jac = vmap(jacobian(jacobian(jacobian(mycov,1),1),0),0)
hess_jac2 = vmap(jacobian(jacobian(jacobian(mycov,0),0),1),0)
hess_hess = vmap(jacobian(jacobian(jacobian(jacobian(mycov,1),1),0),0),0)

In [8]:
def build_cov_composite(example_i, example_j):
  '''example_i: row_index of covariance matrix, (N,2) for N points,
  second column required to be 0,1,2 for function, first-deriv, second-deriv
  must be in order, all 0s first, then 1s, then 2s. (to exploit upper triangular structure)
    example_j: col-index of covariance matrix, (M,2) for M points
    same specifications for the second column as example_i
  '''
  #if samearr we will get upper-tri and then make it symmetric. otherwise we will
  #get the whole matrix.
  samearr = np.array_equal(example_i, example_j)
  N = len(example_i)
  M = len(example_j)
  KK = np.zeros((N,M))
  inds0i = np.where(example_i[:,1]==0)[0]
  inds1i = np.where(example_i[:,1]==1)[0]
  inds2i = np.where(example_i[:,1]==2)[0]

  inds0j = np.where(example_j[:,1]==0)[0]
  inds1j = np.where(example_j[:,1]==1)[0]
  inds2j = np.where(example_j[:,1]==2)[0]

  def getinds(indsi, indsj, utri=True): 
    """returns 
    inds: tuple of ((i), (j)) index for covariance matrix
    points: array of (2,x). row and column values to use for calculating covar
    """  
    x,y = np.meshgrid(indsi,indsj)
    if utri:
      g = [(xi,yi) for xi,yi in zip(x.flatten(), y.flatten()) if yi>=xi]
      inds = tuple(zip(*g))
    else:
      inds = tuple((x.flatten(),y.flatten()))
    ival = example_i[inds[0],0]
    jval = example_j[inds[1],0]
    points = np.vstack((ival,jval))
    return inds, points 

  utri=samearr

  #fill in (f,f) covariance
  if len(inds0i) != 0 and len(inds0j) != 0:
    inds, points = getinds(inds0i, inds0j, utri)
    KK[inds] = mycov(points[0], points[1])


  #fill in (f,f') covariance
  if len(inds0i) != 0 and len(inds1j) != 0:
    inds, points = getinds(inds0i, inds1j, utri)
    KK[inds] = v_grad(points[0], points[1])
    if not samearr:
      if len(inds1i) != 0 and len(inds0j) != 0:
        inds, points = getinds(inds1i, inds0j, utri)
        KK[inds] = v_grad2(points[0], points[1])


  #fill in (f,f'') covariance
  if len(inds0i) != 0 and len(inds2j) != 0:
    inds, points = getinds(inds0i,inds2j, utri)
    KK[inds] = ffppcov(points[0], points[1])
    if not samearr:
      if len(inds2i) != 0 and len(inds0j) != 0:
        inds, points = getinds(inds2i,inds0j, utri)
        KK[inds] = ffppcov2(points[0], points[1])

  #fill in (f',f') covariance
  if len(inds1i) != 0 and len(inds1j) != 0:
    inds, points = getinds(inds1i,inds1j, utri)
    KK[inds] = fpfpcov(points[0], points[1])

  #fill in (f', f'') covariance
  if len(inds1i) != 0 and len(inds2j) != 0:
    inds, points = getinds(inds1i, inds2j, utri)
    KK[inds] = hess_jac(points[0], points[1])
    if not samearr:
      if len(inds1j) != 0 and len(inds2i) != 0:
        inds, points = getinds(inds2i, inds1j, utri)
        KK[inds] = hess_jac2(points[0], points[1])

  #fill in (f'',f'') covariance
  if len(inds2i) !=0 and len(inds2j) != 0:
    inds, points = getinds(inds2i, inds2j, utri)
    KK[inds] = hess_hess(points[0], points[1])

  if samearr:
    KK = KK + KK.T - np.diag(np.diagonal(KK))
  return KK


In [9]:
def get_mean_var(feature_list, Yn, dom_list, sigma_n):
    '''
    feature_list: train x,
    Yn: train y,
    dom_list: points to sample,
    sigma_n: regularizer (likelihood noise?)
    '''
    Knn = build_cov_composite(feature_list, feature_list) 
    N_samp, _ = np.shape(Knn)
    M_dom = len(dom_list)
    Kmm = build_cov_composite(dom_list, dom_list)
    Knm = build_cov_composite(feature_list, dom_list)
    
    common_term = np.matmul(Knm.T, np.linalg.inv(Knn + 
                                np.eye(N_samp)*sigma_n**2))

    mu = np.matmul(common_term, Yn)
    var = Kmm - np.matmul(common_term, Knm)
    
    return mu, var

In [10]:
#change this per problem, from gpytorch
#for Au
#lengthscale = 4.2634
#outputscale = 0.8007

#for Pd
lengthscale = 2.5931
outputscale = 0.0796

In [12]:
#train input X
x_data = np.vstack((V,np.zeros_like(V))).T

#make test input X*
v_test1 = np.linspace(9., 29., 1000)

v_test = np.vstack((v_test1,np.zeros_like(v_test1)))

v_testp = np.vstack((v_test1, np.ones_like(v_test1)))
v_testpp = np.vstack((v_test1, 2*np.ones_like(v_test1)))

v_testall = np.hstack((v_test, v_testp, v_testpp)).T

In [ ]:
#we used sigma_n = 1e-3 for Au, and used 0.5e-3 for Pd.
#it helps the stability.
sigma_n = 0.5e-3
mu1, var1 = get_mean_var(x_data, nrg, v_testall, sigma_n)
samples2 = np.random.multivariate_normal(mu1, var1, size=1000)


In [ ]:
np.save('mu-au.npy', mu1)
np.save('var-au.npy', np.diagonal(var1))
np.save('samples-au.npy', samples2)